**Project 2 Phase 2**<br>
Group 8
<br>
<table style="width:100%">
  <tr>
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">Christian</td>
    <td style = "text-align: left">Peinthor</td>
    <td style = "text-align: left">11815592</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Michael</td>
    <td style = "text-align: left">Weikl</td>
    <td style = "text-align: left">1154652</td>
  </tr>
  <tr>
    <td style = "text-align: left">3</td>
    <td style = "text-align: left">Uros</td>
    <td style = "text-align: left">Zivanovic</td>
    <td style = "text-align: left">12032271</td>
  </tr>
  <tr>
    <td style = "text-align: left">4</td>
    <td style = "text-align: left">Maximilian</td>
    <td style = "text-align: left">Hofmann</td>
    <td style = "text-align: left">12019987</td>
  </tr>
  
</table>
<br>

# Basic stuff

In [1]:
import numpy as np
import numpy.linalg as lin
import scipy.optimize

# Implementations

## Simplex

In [2]:
def simplex_method(linear_problem: np.ndarray, constraints: np.ndarray, constraint_rhs: np.ndarray):
    def swap(variables: list, exits_idx: int, enters_idx: int) -> None:
        """
        In-/excluding respective vectors into/from the basis.
        """
        tmp = variables[exits_idx]
        variables[exits_idx] = variables[enters_idx]
        variables[enters_idx] = tmp

    base_size = constraints.shape[0]  # Number of basis vectors
    non_basis_count = constraints.shape[1]  # Number of non basis vectors
    working_matrix = np.concatenate((constraints, np.eye(base_size)), axis=1)  # Extended matrix of the system
    x = np.concatenate((np.zeros([non_basis_count, 1]), constraint_rhs))  # Right parts of the constraints
    weights = np.concatenate((linear_problem, np.zeros([base_size, 1])))  # Objective function quotients
    variable_idx = list(range(len(weights)))  # Indices for all variables

    while True:
        basis_idx = variable_idx[non_basis_count:]
        basis = working_matrix.T[basis_idx].T
        basis_inv = lin.inv(basis)
        x_B = x[basis_idx]
        c_B = weights[basis_idx]
        lam = lin.inv(basis.T) @ c_B

        non_basis_idx = variable_idx[:non_basis_count]
        c_N = weights[non_basis_idx]
        n_T = working_matrix.T[non_basis_idx]
        s_N = c_N - (n_T @ lam)

        if np.all(s_N >= 0):
            print("Optimal solution found!!!")
            solution = c_B.T @ basis_inv @ constraint_rhs
            return x_B, solution

        # since here must be at least one s_q < 0, we take the argmin to get the index of the smallest value.
        q_idx = np.argmin(s_N)  # Fetch the idx in the non basis idx set
        q = non_basis_idx[q_idx]  # Fetch the variable_idx
        d = basis_inv @ working_matrix.T[q].T
        d = np.reshape(d, (len(d), 1))

        if np.any(d <= 0):
            print("Problem is unbound!!!")
            return None, None

        piv_weigths = x_B / d
        p_idx = np.argmin(piv_weigths)
        x_q = piv_weigths[p_idx]
        x[basis_idx] = x_B - (d * x_q)
        x[q] = x_q  # set entering variable vaLue

        # In-/excluding respective vectors into/from the basis:
        swap(variables=variable_idx, exits_idx=p_idx + non_basis_count, enters_idx=q_idx)

## Active Set

In [3]:
def compute_learning_rate(A, p, x, w, b):
    numerator = b - (A @ x)
    denominator = A @ p
    possible_alphas = numerator / denominator

    idx_gtz = np.where(denominator > 0)[0].tolist()

    if len(idx_gtz) == 0:
        return 1, None
    else:
        blocking = []
        for idx in idx_gtz:
            if idx not in w:
                blocking.append(idx)

        if len(blocking) == 0:
            return 1, None
        else:
            possible_alphas = possible_alphas[blocking]
            min_alpha = np.argmin(possible_alphas)
            if len(min_alpha.shape) != 0:
                min_alpha = min_alpha[0]

            if possible_alphas[min_alpha] < 1:
                return possible_alphas[min_alpha], min_alpha
            else:
                return 1, None


def compute_descent_direction_and_lagrangien(G, x, c, A, w):
    working_A = A[w]
    size_of_zeros = working_A.shape[0]

    problem = np.concatenate((
        np.concatenate((G, working_A.T), axis=1),
        np.concatenate((working_A, np.zeros((size_of_zeros, size_of_zeros))), axis=1)),
        axis=0)
    g_k = (G @ x) + c
    rhs = np.concatenate((-g_k, np.zeros(size_of_zeros)), axis=0)

    result = np.linalg.solve(problem, rhs.T)

    descent_direction = result[0:len(G)]
    lagrangiens = result[len(G):]

    return descent_direction, lagrangiens


def active_set_method(x_0, G, c, A, b, steps=1000):
    x = x_0
    w: list = np.where(A @ x == b)[0].tolist()

    for k in range(steps + 1):
        p, lambdas = compute_descent_direction_and_lagrangien(G=G, x=x, c=c, A=A, w=w)

        if np.all(np.abs(p) <= 1e-15):
            if np.all(lambdas >= 0):
                return x, k
            else:
                j = np.argmin(lambdas)
                w_j = w[j]
                while w_j in w:
                    w.remove(w_j)

        else:
            alpha, blocking_idx = compute_learning_rate(A, p, x, w, b)
            x = x + alpha * p

            # Are there blocking constraints
            # http://lendek.net/teaching/OPT/qp.pdf
            if alpha < 1:
                w.append(blocking_idx)

    return None, k

## SQP

In [4]:
### Implementation

# Testruns

## Simplex

In [5]:
prob = np.array([[-3], [-2]])
const = np.array([[1, 1], [2, 0.5]])
rhs = np.array([[5], [8]])

x_val, sol = simplex_method(linear_problem=prob, constraints=const, constraint_rhs=rhs)
print(x_val)
print(sol)

Optimal solution found!!!
[[1.33333333]
 [3.66666667]]
[[-13.66666667]]


## Active Set

In [9]:
G = np.array([[-2, 0], [0, -2]])
c = np.array([6, 4])
A = np.array([[1, 1], [-1, 0], [0, -1]])
b = np.array([3, 0, 0])

result, iterations = active_set_method(np.array([0, 1]), G, c, A, b)
print(f"Found solution at {result} after {iterations} iterations")

Found solution at [0. 2.] after 1 iterations


C:\Users\weikl\AppData\Local\Temp\ipykernel_24492\2068304034.py:4: RuntimeWarning: invalid value encountered in true_divide
  possible_alphas = numerator / denominator


## SQP